In [1]:
from keystone import *
from unicorn import *
from unicorn.x86_const import *
import pandas as pd
from tqdm import tqdm
from IPython.core.debugger import set_trace
from capstone import *

md = Cs(CS_ARCH_X86, CS_MODE_64)
ks = Ks(KS_ARCH_X86, KS_MODE_64)

In [8]:
# memory address where emulation starts
ADDRESS = 0x0
code=bytes.fromhex("48b8d8cfce84dac3c6c648890748b8ead9ddc3d9d9c9c54889470848b8c784c9c5c7000000488947104831c98a040f34aa88040f48ffc14883f91575ef")

In [4]:
for i in md.disasm(code, 0x1000):
    print("0x%x:\t%s\t%s" %(i.address, i.mnemonic, i.op_str))

0x1000:	movabs	rax, 0xc6c6c3da84cecfd8
0x100a:	mov	qword ptr [rdi], rax
0x100d:	movabs	rax, 0xc5c9d9d9c3ddd9ea
0x1017:	mov	qword ptr [rdi + 8], rax
0x101b:	movabs	rax, 0xc7c5c984c7
0x1025:	mov	qword ptr [rdi + 0x10], rax
0x1029:	xor	rcx, rcx
0x102c:	mov	al, byte ptr [rdi + rcx]
0x102f:	xor	al, 0xaa
0x1031:	mov	byte ptr [rdi + rcx], al
0x1034:	inc	rcx
0x1037:	cmp	rcx, 0x15
0x103b:	jne	0x102c


In [13]:
def read_string(uc, address):
    ret = ""
    c = uc.mem_read(address, 1)[0]
    read_bytes = 1

    while c != 0x0:
        ret += chr(c)
        c = uc.mem_read(address + read_bytes, 1)[0]
        read_bytes += 1
    return ret

def emulate(code):

    try:
        # Initialize emulator
        mu = Uc(UC_ARCH_X86, UC_MODE_64)

        # map 2MB memory for this emulation
        mu.mem_map(ADDRESS, 2 * 1024 * 1024)

        # write machine code to be emulated to memory
        mu.mem_write(ADDRESS, code)

        # initialize stack
        mu.reg_write(UC_X86_REG_ESP, ADDRESS + 0x100000)

        # tracing all basic blocks with customized callback
        #mu.hook_add(UC_HOOK_BLOCK, hook_block)

        # tracing all instructions with customized callback
        #mu.hook_add(UC_HOOK_CODE, hook_code)

        # handle interrupt ourself
        #mu.hook_add(UC_HOOK_INTR, hook_intr)

        # handle SYSCALL
        #mu.hook_add(UC_HOOK_INSN, hook_syscall32, None, 1, 0, UC_X86_INS_SYSCALL)

        # set key candiate to ESI
        #mu.reg_write(UC_X86_REG_ESI, key_canidate)
        
        # emulate machine code in infinite time
        mu.emu_start(ADDRESS, ADDRESS + len(code))

        # now print out some registers
        print(">>> Emulation done")

        print(read_string(mu, 0))
        
    except UcError as e:
        print("ERROR: %s" % e)

In [14]:
emulate(code)

>>> Emulation done
red.pill@swisscom.com
